# This notebook needs fixing. The loss calculations don't seem accurate at all. They are low before starting training and the test vs train losses are very different
Could is be that some kind of correction for batch size is happening in one case and not the other

In [1]:
from neural_network import *
from activation_functions import *
from loss_functions import *
from layers import *

First I will get it working with 1 input neuron and one output neuron

In [2]:
traning_cycles_one_neuron = 100_000
training_reports_one_neuron = 100
learning_rate_one_neuron = 10**-2
num_input_neurons_one_neuron = 1
batch_size_one_neuron = 50
test_data_items_one_neuron = 10_000
x_range_one_neuron = [-1.0, 1.0]

In [3]:
def network_factory_one_neuron():
    return Neural_Network(
        num_input_neurons = num_input_neurons_one_neuron,
        loss_function = MC_MSE(),
        batch_size = batch_size_one_neuron,
        learning_rate = learning_rate_one_neuron,
        layers = [
            Neural_Layer(1, Identity_Function()),
        ]
    )\
        .set_full_validation(False)\
        .compile()

In [4]:
target_network_one_neuron = network_factory_one_neuron()
training_network_one_neuron = network_factory_one_neuron()

In [5]:
# heart surgery so provate attributes accessed
target_network_one_neuron._layers[0]._weights_m = np.array([[3.0]])
target_network_one_neuron._layers[0]._bias_v = np.array([5.0])


In [6]:
def run_test_one_neuron(echo=True):
    X_data = np.random.uniform(
        low = x_range_one_neuron[0],
        high = x_range_one_neuron[1],
        size = (num_input_neurons_one_neuron, test_data_items_one_neuron)
    )
    if echo: print("Making predictions")
    Y_data = target_network_one_neuron.make_predicitons(X_data)


    if echo: print("Evaluating test loss")
    loss = training_network_one_neuron.evaluate_model_on_test_data(
        X_test_data=X_data,
        Y_test_data=Y_data,
    )
    
    if echo: print({"loss": loss})
    if echo: training_network_one_neuron.get_layers()[0].print_parameters()
    return loss

In [7]:
# target parameters to learn
target_network_one_neuron.get_layers()[0].print_parameters()

Weights (1, 1):   [[3.]]
Bias (1,):   [5.]


In [8]:
training_network_one_neuron.get_layers()[0].print_parameters()

Weights (1, 1):   [[0.25356534]]
Bias (1,):   [0.1]


In [9]:
loss = run_test_one_neuron()


Making predictions
Evaluating test loss
{'loss': 26.767345641520517}
Weights (1, 1):   [[0.25356534]]
Bias (1,):   [0.1]


In [10]:
def train_model_one_neuron():
    X_data = np.random.uniform(
        low = x_range_one_neuron[0],
        high = x_range_one_neuron[1],
        size = (num_input_neurons_one_neuron, batch_size_one_neuron)
    )
    Y_data = target_network_one_neuron.make_predicitons(X_data)

    loss = training_network_one_neuron.train_model_on_minibatch(
        X_train_data=X_data,
        Y_train_data=Y_data,
    )
    return loss, X_data, Y_data

In [11]:
loss, X_data, Y_data = train_model_one_neuron()
print(loss)
print(X_data)
print(Y_data)

30.060717741997628
[[ 0.2750048   0.20977479  0.54836169  0.70121513 -0.44378931 -0.57306909
   0.17386175  0.61134561 -0.68351729 -0.72211697  0.9349859   0.9153632
   0.37255358 -0.48529003 -0.60567327  0.29386213  0.01123997  0.79583484
   0.99572664 -0.7420547   0.55103454 -0.53413299  0.75296516  0.89759862
   0.83762504  0.85560147  0.77797966  0.03559162  0.54702893 -0.40085846
   0.97031347  0.16017895  0.26030893 -0.02047356  0.02695048 -0.11536801
  -0.71810513  0.1498705   0.22671424  0.03880312  0.18413098 -0.82557459
  -0.30096012 -0.86412315  0.74141261 -0.34534715 -0.59018891  0.34631113
   0.73133873 -0.52568942]]
[[5.82501439 5.62932438 6.64508506 7.1036454  3.66863208 3.28079273
  5.52158525 6.83403683 2.94944812 2.83364909 7.80495771 7.74608961
  6.11766073 3.54412992 3.18298019 5.8815864  5.03371991 7.38750452
  7.98717991 2.77383589 6.65310362 3.39760102 7.25889549 7.69279587
  7.51287513 7.56680442 7.33393898 5.10677487 6.64108679 3.79742461
  7.91094042 5.4805368

In [12]:
# should be all zeros if foweward propagation correct
print(3*X_data+5 - Y_data)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [13]:
for traning_cycles_num in range(traning_cycles_one_neuron):
    try: 
        loss, _, _ = train_model_one_neuron()
    except ParameterChangeZero:
        break
    
    if traning_cycles_num % (traning_cycles_one_neuron // training_reports_one_neuron) == 0:
        print(loss)


26.198573102318306
10.779365196312233
4.838290847839083
2.1729882510416703
0.9772451935192012
0.43767900158622186
0.1967547642005062
0.0883717704870403
0.03960339287835831
0.017888134098214246
0.008036698670816732
0.003608325676393908
0.0016210922682281194
0.0007265603947348371
0.0003271447504224621
0.00014670729660209132
6.605254228048123e-05
2.966051423934018e-05
1.3328562836862449e-05
5.985841315550082e-06
2.6901157905420157e-06
1.2116133499765828e-06
5.42933383022293e-07
2.438915015018282e-07
1.0973373748490384e-07
4.9212090688945636e-08
2.2118061433471857e-08
9.943442312482498e-09
4.465565813481848e-09
2.0037293829854075e-09
9.008700341488557e-10
4.0550568147355735e-10
1.8179130330017504e-10
8.138139180484622e-11
3.6840759403515496e-11
1.6508684494499882e-11
7.41460660391664e-12
3.3120158719446774e-12
1.4947432240232903e-12
6.739060989752188e-13
3.013358769334962e-13
1.3530249914166533e-13
6.093419371259994e-14
2.735870507308818e-14
1.2283021220017429e-14
5.50422263593039e-15
2.48

In [14]:
run_test_one_neuron()

Making predictions
Evaluating test loss
{'loss': 4.252906513238618e-19}
Weights (1, 1):   [[3.]]
Bias (1,):   [5.]


4.252906513238618e-19

In [15]:
# target parameters to learn
target_network_one_neuron.get_layers()[0].print_parameters()

Weights (1, 1):   [[3.]]
Bias (1,):   [5.]


In [16]:
X_data = np.random.uniform(
    low = x_range_one_neuron[0],
    high = x_range_one_neuron[1],
    size = (num_input_neurons_one_neuron, batch_size_one_neuron)
)
Y_data = target_network_one_neuron.make_predicitons(X_data)

In [17]:
assert (3*X_data + 5 == Y_data).all()

In [18]:
assert target_network_one_neuron.evaluate_model_on_test_data(X_data, Y_data) < 10**-12

In [19]:
loss_test = training_network_one_neuron.evaluate_model_on_test_data(X_data, Y_data)
# loss train is on foreward propagation before model parameters are changed
loss_train = training_network_one_neuron.train_model_on_minibatch(X_data, Y_data)

assert (loss_test == loss_train).all()

Next I will test one layer

In [20]:
traning_cycles_one_layer = 100_000
training_reports_one_layer = 50
learning_rate_one_layer = 10**-2
num_input_neurons_one_layer = 2
num_output_neurons_one_layer = 2
batch_size_one_layer = 50
test_data_items_one_layer = 10_000
x_range_one_layer = [-1.0, 1.0]

In [21]:
def network_factory_one_layer():
    return Neural_Network(
        num_input_neurons = num_input_neurons_one_layer,
        loss_function = MC_MSE(),
        batch_size = batch_size_one_layer,
        learning_rate = learning_rate_one_layer,
        layers = [
            Neural_Layer(num_output_neurons_one_layer, Identity_Function()),
        ]
    )\
        .set_full_validation(False)\
        .compile()

In [22]:
training_network_one_layer = network_factory_one_layer()
target_network_one_layer = network_factory_one_layer()

target_network_one_layer._layers[0]._bias_v = np.random.uniform(-1, 1, (num_output_neurons_one_layer,))

In [23]:
target_network_one_layer.get_layers()[0].print_parameters()
training_network_one_layer.get_layers()[0].print_parameters()
training_network_one_layer._num_trains = 0

Weights (2, 2):   [[-0.92750534  0.69491069]
 [-0.71227767 -0.38931715]]
Bias (2,):   [0.13019614 0.70450393]
Weights (2, 2):   [[-0.01826356  0.85679798]
 [-0.16273809 -0.47405338]]
Bias (2,):   [0.1 0.1]


In [24]:
def run_test_one_layer(echo=False):
    X_data = np.random.uniform(
        low = x_range_one_layer[0],
        high = x_range_one_layer[1],
        size = (num_input_neurons_one_layer, test_data_items_one_layer)
    )
    if echo: print("Making predictions")
    Y_data = target_network_one_layer.make_predicitons(X_data)


    if echo: print("Evaluating test loss")
    loss = training_network_one_layer.evaluate_model_on_test_data(
        X_test_data=X_data,
        Y_test_data=Y_data,
    )

    if echo: print({"loss": loss})
    if echo: training_network_one_layer.get_layers()[0].print_parameters()
    return loss


In [25]:
def train_model_one_layer():
    X_data = np.random.uniform(
        low = x_range_one_layer[0],
        high = x_range_one_layer[1],
        size = (num_input_neurons_one_layer, batch_size_one_layer)
    )
    Y_data = target_network_one_layer.make_predicitons(X_data)

    loss = training_network_one_layer.train_model_on_minibatch(
        X_train_data=X_data,
        Y_train_data=Y_data,
    )
    training_network_one_layer._num_trains += 1
    return loss

In [26]:
run_test_one_layer()

0.3786895663226945

In [27]:
train_model_one_layer()

0.39143219647684246

In [28]:
run_test_one_layer()

0.38079088755606505

In [29]:
for _ in range(100):
    train_model_one_layer()

In [30]:
run_test_one_layer()

0.36901440445061817

In [31]:
for traning_cycles_num in range(traning_cycles_one_layer):
    try: 
        loss = train_model_one_layer()
    except ParameterChangeZero:
        break
    
    if traning_cycles_num % (traning_cycles_one_layer // training_reports_one_layer) == 0:
        print([training_network_one_layer._num_trains, loss])
print(loss)

[102, 0.32731231834508606]
[2102, 0.31352543674267386]
[4102, 0.16910597169102]
[6102, 0.15413160045145835]
[8102, 0.15754670626172879]
[10102, 0.12945703128810487]
[12102, 0.11284145287385236]
[14102, 0.0959820981238243]
[16102, 0.09058256783681672]
[18102, 0.08266506269917041]
[20102, 0.07480706033862425]
[22102, 0.07150725473020246]
[24102, 0.06237804176711332]
[26102, 0.06343592391555347]
[28102, 0.06049032499987033]
[30102, 0.05514927214690456]
[32102, 0.053195794811605426]
[34102, 0.05053422583387196]
[36102, 0.0483516941325697]
[38102, 0.04659026244342004]
[40102, 0.04593751307277857]
[42102, 0.043343020718192404]
[44102, 0.042107175081049636]
[46102, 0.04051733645403962]
[48102, 0.039130750433390975]
[50102, 0.038017078336873796]
[52102, 0.036344807126439685]
[54102, 0.035598237501347954]
[56102, 0.03451253850053564]
[58102, 0.03349209300286793]
[60102, 0.03221830485440144]
[62102, 0.031020712987075403]
[64102, 0.030041534960413573]
[66102, 0.029233881375416406]
[68102, 0.02820

In [32]:
print("Current loss and parameters")
print(run_test_one_layer())
print("target parameters")
target_network_one_layer.get_layers()[0].print_parameters()
print("learned parameters")
training_network_one_layer.get_layers()[0].print_parameters()

Current loss and parameters
0.01680872415300585
target parameters
Weights (2, 2):   [[-0.92750534  0.69491069]
 [-0.71227767 -0.38931715]]
Bias (2,):   [0.13019614 0.70450393]
learned parameters
Weights (2, 2):   [[-0.92452702  0.69844287]
 [-0.70885559 -0.4000052 ]]
Bias (2,):   [0.25973546 0.57493868]


In [33]:
training_network_one_layer.reset_parameters()

In [34]:
run_test_one_layer()

0.48773359807943256

In [35]:
for traning_cycles_num in range(traning_cycles_one_layer):
    X_data = np.random.uniform(
        low = x_range_one_layer[0],
        high = x_range_one_layer[1],
        size = (num_input_neurons_one_layer, batch_size_one_layer)
    )
    Y_data = target_network_one_layer.make_predicitons(X_data)

    P = training_network_one_layer._layers[0].foreward_propagate(X_data)
    loss = training_network_one_layer._loss_function.compute_loss(P, Y_data)
    loss_grad = training_network_one_layer._loss_function.compute_loss_gradient()
    training_network_one_layer._layers[0].back_propagate(loss_grad)
    training_network_one_layer._layers[0].update_parameters(training_network_one_layer._learning_rate)
    
    if traning_cycles_num % (traning_cycles_one_layer // training_reports_one_layer) == 0:
        print(loss)
print(loss)

0.5645527091016791
0.43756198702601756
0.34805890306366793
0.2632638113543751
0.17583083797532437
0.1724361703718088
0.12863415052180166
0.11742323667934723
0.08235847527676063
0.08813017455024624
0.09080622447624843
0.06704945875950856
0.06765699725424056
0.06264541681573169
0.05989796288540204
0.05675894007734352
0.056631673694254384
0.04914671310201478
0.04789164715882589
0.04493733335710003
0.043867375607305865
0.04231582940057265
0.0410487136760605
0.0417395046208438
0.03969573308049265
0.03800683371827312
0.03462868958062354
0.03538545556765674
0.03391976650241631
0.03321096767163956
0.03218230977213242
0.030965863560060276
0.029394031449675305
0.02887359573483772
0.02870770738719906
0.028344148996911435
0.026685922848671734
0.02590282049762001
0.024555485590851554
0.02323440342429753
0.023400399494145154
0.023226303032892394
0.021694583893449543
0.02113732377362021
0.020413756011577293
0.019434724794287123
0.018914161650902078
0.018178403061489653
0.017946046656067298
0.01729985

In [36]:
traning_cycles_deep = 10_000
training_reports_deep = 100
learning_rate_deep = float(1)
num_input_neurons_deep = 2
batch_size_deep = 50
test_data_items_deep = 10_000
x_range_deep = [-1.0, 1.0]

In [37]:
def network_factory_deep():
    return Neural_Network(
        num_input_neurons = num_input_neurons_deep,
        loss_function = MC_MSE(),
        batch_size = batch_size_deep,
        learning_rate = learning_rate_deep,
        layers = [
            Neural_Layer(2, RELU()),
            Neural_Layer(2, Identity_Function()),
        ]
    )\
        .set_full_validation(False)\
        .compile()

In [38]:
training_network_deep = network_factory_deep()
target_network_deep = network_factory_deep()

# set target bias to random
for i in range(2):
    target_network_deep._layers[i]._bias_v = np.random.uniform(-1, 1, (2,))

In [39]:
def run_test_deep():
    X_data = np.random.uniform(
        low = x_range_deep[0],
        high = x_range_deep[1],
        size = (num_input_neurons_deep, test_data_items_deep)
    )
    print("Making predictions")
    Y_data = target_network_deep.make_predicitons(X_data)


    print("Evaluating test loss")
    loss = training_network_deep.evaluate_model_on_test_data(
        X_test_data=X_data,
        Y_test_data=Y_data,
    )

    print({"loss": loss})


In [40]:
def train_model_deep():
    X_data = np.random.uniform(
        low = x_range_deep[0],
        high = x_range_deep[1],
        size = (num_input_neurons_deep, batch_size_deep)
    )
    Y_data = target_network_deep.make_predicitons(X_data)

    loss = training_network_deep.train_model_on_minibatch(
        X_train_data=X_data,
        Y_train_data=Y_data,
    )
    return loss

In [41]:
def print_parameter_differences_deep():
    for target_layer, training_layer in zip(target_network_deep.get_layers(), training_network_deep.get_layers()):
        if isinstance(target_layer, Trainable_Layer_Interface):
            for key in ("W", "B"):
                print(
                    training_layer.get_parameters()[key] - target_layer.get_parameters()[key]
                )

In [42]:
print_parameter_differences_deep()

[[0.17869707 0.94759158]
 [0.94834767 0.39621516]]
[-0.27515408  0.80346043]
[[ 1.38636122 -0.22483284]
 [-0.21914304  0.09926087]]
[-0.27386357  0.03362043]


In [43]:
print("Current loss")
run_test_deep()

# print("parameter_difference")
# print_parameter_differences()

Current loss
Making predictions
Evaluating test loss
{'loss': 0.1736240123630445}


In [44]:
for traning_cycles_num in range(traning_cycles_deep):
    try:
        loss = train_model_deep()
    except ParameterChangeZero:
        break

    if traning_cycles_num % (traning_cycles_deep // training_reports_deep) == 0:
        print(loss)
print(loss)

0.1818979087718204
0.03335026188826796
0.025779257078660925
0.02660855056070475
0.024484903497284577
0.02126015360664035
0.026126678614263567
0.022740564906420246
0.020890369251238154
0.021474671423243396
0.018448684920250577
0.011650193246648519
0.013108819320361816
0.013258289611250076
0.013712439405844008
0.012664858654059368
0.007681549959673024
0.007156676945701394
0.007515901159912775
0.007420792802567097
0.0069077091408545855
0.006280770737541045
0.0057324615094279975
0.005075715034673248
0.005802573884022808
0.004938194621181189
0.004490512636594225
0.004671205806145351
0.004325669088537053
0.004378798748822347
0.0026805678436054407
0.0027020292813146433
0.004380047823729379
0.004244076007584351
0.004378085312917091
0.0046195232839443545
0.003016338353098147
0.0034973771709153824
0.0033236543214503135
0.00199812372932748
0.003464390744569297
0.0030364802992812663
0.0039366476200090785
0.0029736667303175996
0.0037769702094416057
0.003955602577699355
0.003754609962687186
0.003397

In [45]:
print("Current loss")
run_test_deep()

# print("parameter_difference")
# print_parameter_differences()

Current loss
Making predictions
Evaluating test loss
{'loss': 0.0015841017768401305}
